Algorithm 1  

the idea is do keyword extraction using keyword extractor and the train (naive bayes) the obtained keywords to classify them into following categories:  
1. Brand  
2. Model  
3. Color  
4. Description  
5. Price  

Extracting Keywords

few keyword extraction algorithms  
1. Yake  
2. TextRank  
3. Rake


1. yake keyword extractor

In [26]:
import yake

In [27]:
language = "en"
max_ngram_size = 4
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1

extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize , features=None)

In [58]:
with open('test_1.txt','r') as str_file:
    test1 = str_file.read()

In [59]:
print(test1)

Samsung Galaxy M51 (Electric Blue, 6GB RAM, 128GB Storage) 6 Months Free Screen Replacement for Prime
Visit the Samsung Store
 4.4 out of 5 stars    64,698 ratings | 1000+ answered questions
M.R.P.:	â‚¹28,999.00
Deal Price:	â‚¹19,999.00
You Save:	â‚¹9,000.00 (31%)
Inclusive of all taxes
FREE delivery: July 17 - 21 Details
EMI starts at â‚¹941. No Cost EMI available EMI options 
Save Extra with 5 offers
No Cost EMI: Avail No Cost EMI on select cards for orders above â‚¹3000 DetailsNo Cost EMI: Avail No Cost EMI on select cards for orders above â‚¹3000 Details
Bank Offer: Get Rs. 500 Amazon Gift Card on a minimum purchase of Rs 10000 with HDFC Bank Debit Card EMI transaction DetailsBank Offer: Get Rs. 500 Amazon Gift Card on a minimum purchase of Rs 10000 with HDFCâ€¦Details
See 3 more
Pay on Delivery
Pay on Delivery
	
7 Days Replacement
7 Days Replacement
Amazon Delivered
Amazon Delivered
1 Year Warranty
1 Year Warranty
In stock on July 17, 2021.
Order it now.
Sold by STPL Exclusive Onl

In [30]:
words = extractor.extract_keywords(test1)

for word in words:
    print(word)

('Screen Replacement for Prime', 0.0065612265334789996)
('Months Free Screen Replacement', 0.013664053726728558)
('Free Screen Replacement', 0.017936620523088816)
('Cost EMI', 0.02022928602543202)
('Months Free Screen', 0.025698368014288975)
('Avail No Cost EMI', 0.02816314548611035)
('Screen Replacement', 0.033283951389044567)
('Free Screen', 0.0372689658600935)
('EMI', 0.04229563868105315)
('RAM', 0.046662368937777185)
('Cost', 0.05429375832947112)
('Electric Blue', 0.061383280381713555)
('Avail No Cost', 0.07141536113412626)
('Amazon Gift Card', 0.07245009997144747)
('Prime', 0.07253976825056388)
('Replacement for Prime', 0.07972357406665637)
('Storage', 0.08180781634028998)
('Screen', 0.08192074793150367)
('Details', 0.08594470888841457)
('Days Replacement', 0.09080936517480057)


the keywords we expected and want were brand name, model, price, ram, storage but the extracted words were not as expected.

2. textrank (gensim lib uses textrank algorithm for keyword extraction)

In [3]:
from gensim.summarization import keywords

C:\Users\LYNX\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


In [4]:
print(keywords(test1))

camera
memory
emi
samsung
storage
dual
bank
details
cards
card
delivery
free
year
amazon
fast
exclusive
box
including
includes
touchscreen
blue
plus
related queries
samoled
july
electric


it detected keyowrds better than the yake algorithm. it extracted important words like camera, memory, emi, samsung, dual, blue, electric but failed to extract the combines words we wanted like 'electric blue' , 'dual camera', 'samsung (model)' as the ngram might be 1 and default.

the key extractor can still be used but the other half of the information must be extracted from the text separately by looking at the surroundings og the keywords.

Algorithm 2  

1.To break the entire text mined into list of strings using line break and ',' , 'and', '()', 'or' other words which convey that there is no intrict relation between the words before and after them.   
2. label the list of stings into different categories  
3. train the a text classifier i.e naive bayes classifier to the strings into different classes  

how the model words:  
the entire text is given to the model, then the text is broken down into multiple strings. These strings are then passes into naive bayes classifier and the strings are classified into different classes. a dictionary is output where class as key and list of string related to the as value 


In [3]:
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline 
import random

In [4]:
# importing data
class_dict = {}
for category in os.listdir(os.path.join(os.getcwd(),'data','classified_data')):
    with open(os.path.join(os.getcwd(),'data','classified_data',category),'r') as str_file:
        class_dict[category.replace('.txt','')] = str_file.read()
    
    

In [5]:
data_list = []
for attribute in class_dict.keys():
    txt = class_dict[attribute]
    txt = txt.splitlines()
    for line in txt:
        data_list.append([line,attribute])

In [10]:
random.shuffle(data_list)
data_list


[['You Save:\tâ‚¹5,200.00 ', 'price'],
 ['Fast face unlock and Side Fingerprint sensor | Dual SIM ', 'specs'],
 ['Bank Offer: Get Rs. 500 Amazon Gift Card on a minimum purchase of Rs 10000 with HDFC Bank Debit Card EMI transaction',
  'NR'],
 ['EMI starts at â‚¹941', 'price'],
 ['Samsung Galaxy M51', 'model'],
 ['iPhone 12 Mini with 20W USB C Adapter', 'model'],
 ['iPhone 12 Pro Max with 20W USB C Adapter', 'model'],
 ['Box also includes: Travel adapter, USB Type-C to C, ejection pin and user manual',
  'NR'],
 ['See 2 more', 'NR'],
 ['', 'price'],
 [' No Cost EMI available EMI options ', 'NR'],
 ['Black', 'color'],
 ['Style name: 3GB RAM & 32GB Storage', 'specs'],
 ['M.R.P.:\tâ‚¹14,999.00', 'price'],
 ['New Apple iPhone 12 Pro ', 'model'],
 [' front facing punch hole camera', 'NR'],
 ['Pay on Delivery', 'NR'],
 ['New Apple iPhone 12 Mini ', 'model'],
 ['13MP+2MP Dual rear camera setup', 'NR'],
 [' 6GB RAM', 'specs'],
 ['inch TFT screen with 128x128 pixels resolution; Network: GSM ', '

In [13]:
parameter = []
label = []
for i in data_list:
    parameter.append(i[0])
    label.append(i[1])
    

In [14]:
parameter

['You Save:\tâ‚¹5,200.00 ',
 'Fast face unlock and Side Fingerprint sensor | Dual SIM ',
 'Bank Offer: Get Rs. 500 Amazon Gift Card on a minimum purchase of Rs 10000 with HDFC Bank Debit Card EMI transaction',
 'EMI starts at â‚¹941',
 'Samsung Galaxy M51',
 'iPhone 12 Mini with 20W USB C Adapter',
 'iPhone 12 Pro Max with 20W USB C Adapter',
 'Box also includes: Travel adapter, USB Type-C to C, ejection pin and user manual',
 'See 2 more',
 '',
 ' No Cost EMI available EMI options ',
 'Black',
 'Style name: 3GB RAM & 32GB Storage',
 'M.R.P.:\tâ‚¹14,999.00',
 'New Apple iPhone 12 Pro ',
 ' front facing punch hole camera',
 'Pay on Delivery',
 'New Apple iPhone 12 Mini ',
 '13MP+2MP Dual rear camera setup',
 ' 6GB RAM',
 'inch TFT screen with 128x128 pixels resolution; Network: GSM ',
 '2.4',
 'Pay on Delivery',
 '7 Days Replacement',
 'Visit the Samsung Store',
 '1 Year Warranty',
 '4G+4G',
 'For any product related queries kindly Contact_us on: ',
 'Form factor\tTouchscreen Phone',
 '

In [15]:
label

['price',
 'specs',
 'NR',
 'price',
 'model',
 'model',
 'model',
 'NR',
 'NR',
 'price',
 'NR',
 'color',
 'specs',
 'price',
 'model',
 'NR',
 'NR',
 'model',
 'NR',
 'specs',
 'specs',
 'NR',
 'NR',
 'NR',
 'NR',
 'benefits',
 'specs',
 'NR',
 'NR',
 'NR',
 'price',
 'specs',
 'specs',
 'NR',
 'NR',
 'NR',
 'brand',
 'NR',
 'NR',
 'NR',
 'color',
 'specs',
 'NR',
 'specs',
 'specs',
 'NR',
 'color',
 'color',
 'specs',
 'NR',
 'NR',
 'NR',
 'benefits',
 'NR',
 'NR',
 'brand',
 'price',
 'NR',
 'NR',
 'NR',
 'NR',
 'color',
 'NR',
 'color',
 'NR',
 'NR',
 'model',
 'NR',
 'price',
 'specs',
 'NR',
 'NR',
 'brand',
 'NR',
 'NR',
 'specs',
 'NR',
 'NR',
 'NR',
 'NR',
 'NR',
 'price',
 'color',
 'color',
 'NR',
 'price',
 'model',
 'NR',
 'NR',
 'NR',
 'NR',
 'benefits',
 'NR',
 'color',
 'specs',
 'NR',
 'brand',
 'NR',
 'NR',
 'NR',
 'color',
 'NR',
 'NR',
 'NR',
 'NR',
 'benefits',
 'price',
 'model',
 'specs',
 'model',
 'specs',
 'specs',
 'NR',
 'specs',
 'price',
 'NR',
 'specs'

data for training has been prepared with each string or words tagged with its attribute. 

In [16]:
# model preparation
model = make_pipeline(TfidfVectorizer(),MultinomialNB())
model.fit(parameter,label)



Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [32]:
model.predict(['Brand\tApple',
 'Sold by STPL Exclusive Online and Fulfilled by Amazon',
 'Choice',
 'Colour: Electric Blue',
 'Brand\tApple',
 'Price:\tâ‚¹71,800.00',
 'Style name: 256GB',
 '512GB','brand samsung','Samsung guru A11'])

array(['brand', 'NR', 'NR', 'color', 'brand', 'price', 'specs', 'specs',
       'brand', 'model'], dtype='<U8')

the given examples are predicted correctly

bisection of the mined text into list of strings

In [72]:
def bisection(string):
    temp = string
    for i in ['(',')','[',']','-']:
        txt = temp.replace(i,'\n')
        temp = txt
    txt = txt.splitlines()
    return txt

def collect(parameter,predictions):
    dict = {}
    for i in range(len(parameter)):
        if predictions[i] != 'NR':
            try:
                dict[predictions[i]].append(parameter[i])
            except:
                dict[predictions[i]] = []
                dict[predictions[i]].append(parameter[i])
    return dict


now putting all together

In [74]:
with open('test_1.txt','r') as str_file:
    test1 = str_file.read()
print(test1)


Samsung Galaxy M51 (Electric Blue, 6GB RAM, 128GB Storage) 6 Months Free Screen Replacement for Prime
Visit the Samsung Store
 4.4 out of 5 stars    64,698 ratings | 1000+ answered questions
M.R.P.:	â‚¹28,999.00
Deal Price:	â‚¹19,999.00
You Save:	â‚¹9,000.00 (31%)
Inclusive of all taxes
FREE delivery: July 17 - 21 Details
EMI starts at â‚¹941. No Cost EMI available EMI options 
Save Extra with 5 offers
No Cost EMI: Avail No Cost EMI on select cards for orders above â‚¹3000 DetailsNo Cost EMI: Avail No Cost EMI on select cards for orders above â‚¹3000 Details
Bank Offer: Get Rs. 500 Amazon Gift Card on a minimum purchase of Rs 10000 with HDFC Bank Debit Card EMI transaction DetailsBank Offer: Get Rs. 500 Amazon Gift Card on a minimum purchase of Rs 10000 with HDFCâ€¦Details
See 3 more
Pay on Delivery
Pay on Delivery
	
7 Days Replacement
7 Days Replacement
Amazon Delivered
Amazon Delivered
1 Year Warranty
1 Year Warranty
In stock on July 17, 2021.
Order it now.
Sold by STPL Exclusive Onl

In [78]:
bisect = bisection(test1)
predict = model.predict(bisect)
final_dict = collect(bisect,list(predict))

In [80]:
final_dict

{'model': ['Samsung Galaxy M51 ', 'Model Name\tSamsung Galaxy M51'],
 'specs': ['Electric Blue, 6GB RAM, 128GB Storage',
  'Style name: 6GB RAM, 128GB Storage',
  '6GB RAM, 128GB Storage',
  '8GB RAM, 128GB Storage',
  'Memory Storage Capacity\t128 GB',
  ' Infinity',
  'O display, FHD+ capacitive touchscreen with 1080 x 2400 pixels resolution, Contrast Ratio: 78960:1',
  'Memory, Storage & SIM: 6GB RAM, 128GB internal memory expandable up to 512GB | Dedicated Memory slot',
  'Android v10.0 operating system with 2.2GHz+1.8GHz Qualcomm | SD730G octa core processor',
  'ion battery | 25W USB Type',
  'Fast face unlock and Side Fingerprint sensor | Dual SIM ',
  'nano+nano',
  ' with dual standby and dual VoLTE | 3.5 mm headphone jack'],
 'price': ['M.R.P.:\tâ‚¹28,999.00',
  'Deal Price:\tâ‚¹19,999.00',
  'You Save:\tâ‚¹9,000.00 ',
  'EMI starts at â‚¹941. No Cost EMI available EMI options '],
 'benefits': ['1 Year Warranty',
  '1 Year Warranty',
  '1 year manufacturer warranty for device

the model extraction and classifies the mined data with good level of satisfaction. test evaluation was not performed because the label data is not clean and not properly labeled due to no labeling standards

why naive bayes
it is a simple algorithm which can handle but continous and discrete data. it also learns much better than many learning algorithms when there is lack of data in other words can perfrom good with less training data. it is pretty fast and can be used real time


the same problem can be approched in several ways, algorithm 1 and 2 being one of them do to lack of time and and data algorithm 1 was not chosen as it needed a custom modelled keyword extractor for our need but if done so would perform much better than algorithm 2. the other way is to manually pick keywords like gb,model name, colour and select the words arounding the keywords to be in the specific category. like data has colour and the colour of the phone in the same line, so we find the keyword colour in the mined data and the next word to it is the colour this is a rigid way of classification which doesnt require a learning algorithm. but as the attributes become complex it fails.

the data here was manually label into different category which you can look in the classified folder